# In this notebook
- Randomized iterative methods (Gower's paper)
- Using them to linear solve matrices AX = B
- ADI 

All are tested very loosely with small matrices. A more formal unit test suite can be found in the project in `../src`.

In [ ]:
function rand_linsolve(A,b)
    while true
        ()
    
end